In [2]:
import duckdb
import pandas as pd
import numpy as np
from pathlib import Path

In [10]:
root_path = "../"
dbname = "compe_02"
# root_path = root_path.as_posix() + '/'

# Connect to DuckDB database
con = duckdb.connect(root_path+'duckdb/'+ dbname + ".duckdb")    

meses_train = [202012]
meses_test = [202108]

meses = meses_train + meses_test
meses = ', '.join(map(str, meses))


query = (f"""
        SELECT *
        FROM {dbname}
        WHERE {dbname}.foto_mes IN ({meses});""")

# Execute the join query and fetch the result as a Pandas DataFrame
data = con.execute(query).fetchdf()

con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
def psi(expected, actual, buckets=255):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    # Use qcut to get bin edges
    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop', retbins=True)[1]

    # Calculate breakpoints
    breakpoints = sorted(list(set(bin_edges)))

    # Calculate histogram counts
    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    # Calculate proportions
    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    # Calculate PSI for non-null values
    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    # Calculate PSI for null values if present
    psi_null = 0
    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0:
        expected_null_percentage = expected.isnull().mean()
        actual_null_percentage = actual.isnull().mean()
        psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [21]:
psi_results = []
for column in data.columns:
  print(column)
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = data.loc[data['foto_mes'].isin(meses_train),column]
    print("Train: ", train_variable[:5])
    score_variable =  data.loc[data['foto_mes'].isin(meses_test),column]
    print("Score: ", score_variable[:5])
    psi_value = psi(train_variable, score_variable)
    print("PSI: ", psi_value)
    psi_results.append({'feature': column, 'psi': psi_value})



numero_de_cliente
Train:  0    249221109
1    249221468
2    249223005
3    249228180
4    249232117
Name: numero_de_cliente, dtype: int64
Score:  66    249221109
67    249221468
68    249223005
69    249228180
70    249232117
Name: numero_de_cliente, dtype: int64
PSI:  0.0024156938614492292
foto_mes
active_quarter
Train:  0    1
1    1
2    1
3    1
4    1
Name: active_quarter, dtype: int64
Score:  66    1
67    1
68    1
69    1
70    1
Name: active_quarter, dtype: int64
PSI:  0.0
cliente_vip
Train:  0    0
1    0
2    0
3    0
4    0
Name: cliente_vip, dtype: int64
Score:  66    0
67    0
68    0
69    0
70    0
Name: cliente_vip, dtype: int64
PSI:  0.0
internet
Train:  0    0
1    0
2    0
3    0
4    0
Name: internet, dtype: int64
Score:  66    0
67    0
68    0
69    0
70    0
Name: internet, dtype: int64
PSI:  0.00015371111403390342
cliente_edad
Train:  0    61
1    53
2    48
3    66
4    79
Name: cliente_edad, dtype: int64
Score:  66    62
67    53
68    49
69    66
70    80
N

C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: invalid value encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


PSI:  0.08031668860227878
mrentabilidad_annual
Train:  0    20260.41
1    17708.07
2    38939.21
3     2504.55
4    31279.33
Name: mrentabilidad_annual, dtype: float64
Score:  66    16161.50
67    36911.77
68    38080.07
69     2691.40
70    35718.76
Name: mrentabilidad_annual, dtype: float64
PSI:  0.056226985136791814
mcomisiones
Train:  0    1606.90
1    1011.84
2     706.54
3    -931.15
4    1665.82
Name: mcomisiones, dtype: float64
Score:  66    3022.92
67    5829.64
68    1146.31
69    2190.03
70     573.10
Name: mcomisiones, dtype: float64
PSI:  0.36914819420369904
mactivos_margen
Train:  0    -290.10
1   -2382.08
2     177.18
3    -202.67
4    -812.79
Name: mactivos_margen, dtype: float64
Score:  66   -3778.86
67     137.00
68      87.60
69    -486.62
70    -614.69
Name: mactivos_margen, dtype: float64
PSI:  0.019204553492331092
mpasivos_margen
Train:  0     609.39
1    1226.95
2     835.21
3      95.06
4     951.75
Name: mpasivos_margen, dtype: float64
Score:  66    1326.34
67 

C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


Score:  66     4090.25
67        0.00
68    36148.78
69        0.00
70        0.00
Name: mautoservicio, dtype: float64
PSI:  0.0194806536146601
ctarjeta_visa
Train:  0    1
1    1
2    1
3    1
4    1
Name: ctarjeta_visa, dtype: int64
Score:  66    1
67    1
68    1
69    1
70    1
Name: ctarjeta_visa, dtype: int64
PSI:  0.0002517388345040993
ctarjeta_visa_transacciones
Train:  0    14
1    17
2    16
3     5
4    11
Name: ctarjeta_visa_transacciones, dtype: int64
Score:  66    18
67    46
68    17
69     8
70    12
Name: ctarjeta_visa_transacciones, dtype: int64
PSI:  0.002971997376619953
mtarjeta_visa_consumo
Train:  0    36181.41
1    35583.77
2    10776.15
3    10357.64
4    43610.75
Name: mtarjeta_visa_consumo, dtype: float64
Score:  66    108747.66
67    103901.34
68     25441.72
69     24909.18
70     56655.40
Name: mtarjeta_visa_consumo, dtype: float64
PSI:  0.03960444111804311
ctarjeta_master
Train:  0    1
1    1
2    0
3    1
4    1
Name: ctarjeta_master, dtype: int64
Score:

...

C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: invalid value encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


Score:  66       NaN
67       NaN
68    7741.8
69       NaN
70       NaN
Name: m_promedio_cheques_emitidos, dtype: float64
PSI:  inf
m_promedio_cheques_depositados_rechazados
Train:  0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: m_promedio_cheques_depositados_rechazados, dtype: float64
Score:  66   NaN
67   NaN
68   NaN
69   NaN
70   NaN
Name: m_promedio_cheques_depositados_rechazados, dtype: float64
PSI:  nan
m_promedio_cheques_emitidos_rechazados
Train:  0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: m_promedio_cheques_emitidos_rechazados, dtype: float64
Score:  66   NaN
67   NaN
68   NaN
69   NaN
70   NaN
Name: m_promedio_cheques_emitidos_rechazados, dtype: float64
PSI:  nan
m_promedio_atm
Train:  0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: m_promedio_atm, dtype: float64
Score:  66          NaN
67          NaN
68    19207.875
69          NaN
70          NaN
Name: m_promedio_atm, dtype: float64
PSI:  0.05235735942335547
m_promedio_atm_other
Train:  0   NaN
1   NaN
2   NaN
3   NaN
4 

C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: invalid value encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in divide
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\Ap

PSI:  0.03131335132374326
proporcion_limite_visa_cubierto
Train:  0    0.059749
1   -0.121388
2    0.119183
3    0.047116
4    0.377327
Name: proporcion_limite_visa_cubierto, dtype: float64
Score:  66    0.151841
67    0.008426
68    0.133329
69    0.110846
70    0.496074
Name: proporcion_limite_visa_cubierto, dtype: float64
PSI:  0.03711173788697459
proporcion_financiacion_total_cubierto
Train:  0    0.055201
1   -0.027411
2    0.132426
3    0.034900
4    0.172633
Name: proporcion_financiacion_total_cubierto, dtype: float64
Score:  66    0.150853
67    0.050060
68    0.148143
69    0.082108
70    0.226962
Name: proporcion_financiacion_total_cubierto, dtype: float64
PSI:  0.03090702753039505
proporcion_limite_total_cubierto
Train:  0    0.049673
1   -0.024666
2    0.119183
3    0.031410
4    0.155370
Name: proporcion_limite_total_cubierto, dtype: float64
Score:  66    0.135748
67    0.045048
68    0.133329
69    0.073898
70    0.204266
Name: proporcion_limite_total_cubierto, dtype: flo

C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\jgjua\AppData\Local\Temp\ipykernel_2148\2510243848.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)


In [25]:
psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)
# print full table
# Set display options to show the full table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Print the full table
print(psi_df)

                                               feature           psi
232                     tc_proporcion_adelanto_dolares           inf
200                         m_promedio_pagodeservicios           inf
160                       mtc_financiacionlimite_total           inf
140                                 Visa_mlimitecompra           inf
165                             mtc_limitecompra_total           inf
181                              tc_fvencimiento_mayor           inf
182                              tc_fvencimiento_menor           inf
137                                 Visa_msaldodolares           inf
133                                  Visa_Fvencimiento           inf
132                          Visa_mfinanciacion_limite           inf
126                               Master_mconsumototal           inf
195  m_promedio_tarjeta_master_consumo_por_transaccion           inf
197                    m_promedio_prestamos_prendarios           inf
153                               

In [ ]:

def preparar_data(dbname, mes_train, mes_test, drop_cols=None, sampling = 1):

    print('Preparando data...')

 

    data = data.drop(['numero_de_cliente_1', 'foto_mes_1', 'clase_ternaria_1'], axis=1)

    # data = pd.read_parquet(dataset_path + dataset_file)

    data['clase_peso'] = 1.0

    data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
    data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

    data['clase_binaria1'] = 0
    data['clase_binaria2'] = 0
    data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
    data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

#   data = data.loc[data['foto_mes'].isin(mes_train + mes_test)]

    data_continua = data[data['clase_ternaria'] == 'CONTINUA']

    data_continua = data_continua.groupby('foto_mes').sample(frac=sampling, random_state=42)

    data = pd.concat([data_continua, data[data['clase_ternaria'] != 'CONTINUA']])

    print(data.shape)

    print(data.head())

    del data_continua

    train_data = data.loc[data['foto_mes'].isin(mes_train)]
    test_data = data[data['foto_mes'].isin(mes_test)]

    X_train = train_data.drop(['numero_de_cliente','clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_train_binaria1 = train_data['clase_binaria1']
    y_train_binaria2 = train_data['clase_binaria2']
    w_train = train_data['clase_peso']

    X_test = test_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)
    y_test_binaria1 = test_data['clase_binaria1']
    y_test_class = test_data['clase_ternaria']
    w_test = test_data['clase_peso']

    print('Data preparada')

    return X_train, y_train_binaria1, y_train_binaria2, w_train, X_test, y_test_class, y_test_binaria1, w_test


